## Importing Dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import pickle as pkl
import cv2
import h5py

from preprocess_data import dataloader
from siamese_train import siamese_network
from mpl_toolkits.axes_grid1 import ImageGrid
import re

from tqdm import tqdm
from math import ceil

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from skimage.transform import rotate, AffineTransform, warp, rescale
from skimage.util import random_noise

import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Lambda, Input, Flatten, Dense, Concatenate, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import model_from_json
from scipy.interpolate import make_interp_spline, BSpline
import tensorflow.keras.backend as K

In [2]:
np.random.seed(0)
random.seed(0)
tf.random.set_seed(0)

## Plot Training Accuracy and Training Loss


In [3]:
def plot_metric(loss, acc):
    
    size = len(acc)
    x = np.array(range(1,size+1))
    xnew = np.linspace(1,size,100)

    spl = make_interp_spline(x, acc, k=3) #BSpline object
    ynew = spl(xnew)
    plt.figure(figsize=(20,10))
    plt.subplot(1,2,1)
    plt.plot(acc,color = 'b', alpha=0.4, label = 'Avg Batch Accuracy')
    plt.plot(xnew, ynew,color = 'b', alpha=1, label = 'Smooth Average Accuracy')
    plt.xlabel('Steps (for every 500)')
    plt.ylabel('Accuracy')
    plt.legend(loc = "upper left")
    plt.title('Accuracy')

    spl = make_interp_spline(x, loss, k=3) #BSpline object
    ynew = spl(xnew)
    plt.subplot(1,2,2)
    plt.plot(loss,color = 'b', label = 'Avg Batch Loss')
    #plt.plot((xnew, ynew),color = 'b', alpha=1)
    plt.xlabel('Steps (for every 500)')
    plt.ylabel('Loss')
    plt.legend(loc = "upper left")
    plt.title('Loss')

In [4]:
# os.path.join(save_path,"val.pickle")
#print(os.path.getsize('val_acc'))
with open('val_acc', 'r') as f:
    v_acc,train_metrics = pkl.load(f)


FileNotFoundError: [Errno 2] No such file or directory: 'val_acc'

In [ ]:
train_metrics = np.array(train_metrics)
plot_metric(train_metrics[:,0],train_metrics[:,1])

## Plotting Validation Accuracy on 20 - way one shot

In [ ]:
v_acc  =np.array(v_acc)

x = np.array(range(1,len(v_acc)+1))
xnew = np.linspace(1,len(v_acc),100)

spl = make_interp_spline(x, v_acc[:,0], k=3) #BSpline object
ynew1 = spl(xnew)
spl = make_interp_spline(x, v_acc[:,1], k=3) #BSpline object
ynew2 = spl(xnew)
plt.figure(figsize=(10,10))
plt.plot(v_acc[:,0], color = 'b', alpha=0.4,  label = 'within alphabet pairs')
plt.plot(xnew, ynew1, color = 'b', alpha=1, label = 'smoothed within alphabet pairs')
plt.plot(v_acc[:,1], color = 'r', alpha=0.4,  label = 'unstructed alphabets pairs')
plt.plot(xnew, ynew2, color = 'r', alpha=1, label = 'smoothed unstructed alphabets pairs')
plt.xlabel('Steps (for every 500)')
plt.ylabel('Accuracy')
plt.legend(loc = "best")
plt.title('20-way Accuracy')

In [ ]:
train_metrics[-1]


In [ ]:
v_acc[-1]


# Testing N- way one shot

## Loading Saved Model

In [ ]:
siamese = siamese_network()
siamese.get_model()

In [ ]:
best_model = 'best_model/best_model.h5'
siamese.model.load_weights(best_model)
siamese_net = siamese.model
print("Model loaded from disk")

## Evaluation on 10 Alphabets ( Evaluation Set)

In [ ]:

wA_file ='wA_eval_10_split_images.pkl'
uA_file ='uA_eval_10_split_images.pkl'

wA_acc, uA_acc = siamese.test_validation_acc(wA_file, uA_file)

In [ ]:
print("Within Alphabet pairs Accuracy for 20-way one shot samples : {}%".format(wA_acc))
print("Unstructured Alphabet pairs Accuracy for 20-way one shot samples : {}%".format(uA_acc))

## Evaluation on 20 Alphabets ( Image Evaluation Folder)

In [ ]:
wA_file ='wA_eval_20_split_images.pkl'
uA_file ='uA_eval_20_split_images.pkl'

wA_acc, uA_acc = siamese.test_validation_acc(wA_file, uA_file)

print("Within Alphabet pairs Accuracy for 20-way one shot samples : {}%".format(wA_acc))
print("Unstructured Alphabet pairs Accuracy for 20-way one shot samples : {}%".format(uA_acc))

## N-way one shot Testing

In [ ]:
from preprocess_data import dataloader
import os 

folder_path = 'images_evaluation'
dir_list = os.listdir(folder_path)
dl = dataloader()

In [ ]:
def nearest_neighbour_correct(pairs,targets):
    """returns 1 if nearest neighbour gets the correct answer for a one-shot task
        given by (pairs, targets)"""
    X_left, X_right = pairs
    L2_distances = np.zeros_like(targets)
    for i in range(len(targets)):
        L2_distances[i] = np.sqrt(abs(np.sum(X_left[i]**2 - X_right[i]**2)))
    if np.argmin(L2_distances) == np.argmax(targets):
        return 1
    return 0

In [ ]:
def test_data_pairs(n_way = 20, wA = True):
    if wA:
        pairs = dl.wA_test_pairs(folder_path = folder_path, dirs = dir_list, savefilename =None, n_way = n_way)
    else:
        pairs = dl.uA_test_pairs(folder_path = folder_path, dirs = dir_list, savefilename =None, classes = None, n_way = n_way)
    
    X,y =pairs
    correct_pred = 0
    nn_correct = 0
    j = 0
    for i in range(0,len(X),n_way):
        X_left, X_right,_y  = X[i: i+n_way,0],X[i: i+n_way,1], y[i : i+n_way]
        X_left, X_right, _y = np.array(X_left), np.array(X_right), np.array(_y)

        correct_pred += siamese.test_one_shot(X_left, X_right, _y)
        nn_correct += nearest_neighbour_correct((X_left, X_right), _y)

    acc =  correct_pred*100/(len(X)/n_way)
    nn_acc = nn_correct*100/(len(X)/n_way)
    return acc, nn_acc

In [ ]:
def one_shot_accuracy():
    within_accuracies = []
    unstructred_accuracies = []
    #[2, 5, 6, 10, 15, 16, 20]
    for i in tqdm(range(2, 21)):
        within_accuracies.append(test_data_pairs(n_way = i, wA = True))
        unstructred_accuracies.append(test_data_pairs(n_way = i, wA = False)) 
    return within_accuracies ,unstructred_accuracies

In [ ]:
within_accuracies ,unstructred_accuracies = one_shot_accuracy()


In [ ]:
ways = np.arange(2, 21, 1)

plt.figure(figsize = (10,10))
plt.plot(ways,np.asarray(within_accuracies)[:,0], color = 'b',  label = 'wA 20-way one shot accuracy')
plt.plot(ways,np.asarray(unstructred_accuracies)[:,0], color = 'r', label = 'uA 20-way one shot accuracy')
plt.plot(ways,np.asarray(within_accuracies)[:,1], color = 'g',  label = 'wA 20-way nearest neighbour accuracy')
plt.plot(ways,np.asarray(unstructred_accuracies)[:,1], color = 'm', label = 'uA 20-way nearest neighbour accuracy')
plt.xlabel('Steps (for every 500)')
plt.xticks(np.arange(2, 22, step=2))
plt.ylabel('N-way Accuracy')
plt.legend(loc = "best")
plt.title('N-way Accuracies')

In [ ]:
print(within_accuracies)


In [ ]:
print(unstructred_accuracies)


## Visualizing N-way pairs

In [ ]:
def generate_img_matrix(X_left, X_right, y):
    X_left, X_right, _y = np.array(X_left), np.array(X_right), np.array(y)
    pred = siamese_net.predict([X_left, X_right])
    index = np.argmax(pred)
    
    img0 = np.squeeze(X_left[0], axis = 2)
    X_p = []
    img_matrix = []
    
    for i in range(len(X_right)):
        img1 = np.squeeze(X_right[i], axis = 2)
        X_p.append(img1)
        if len(X_p) == 5:
            X_p =np.vstack(X_p)
            img_matrix.append(X_p)
            X_p = []
            
    img_matrix = np.asarray(img_matrix)
    img_matrix = np.hstack(img_matrix)
    return img0, img_matrix, index

In [ ]:
def visualize_n_way(file, n_way = 20):

    with open(file,'rb') as f:
        X,y = pkl.load(f)

    i = random.randint(0,int(len(X)/n_way))

    X_left, X_right,_y  = X[i: i+n_way,0],X[i: i+n_way,1], y[i : i+n_way]
    img0, img_matrix, index= generate_img_matrix(X_left, X_right, _y)

    f, ax=  plt.subplots(1,3, figsize = (20,20))
    f.tight_layout()
    ax[0].imshow(img0, cmap = 'gray')
    ax[0].set_title('Test Image')
    ax[1].imshow(img_matrix, cmap = 'gray')
    ax[1].set_title('Support Set')
    ax[2].imshow(np.squeeze(X_right[index], axis = 2), cmap = 'gray')
    ax[2].set_title('Image with highest similarity in Support Set')

In [ ]:
wA_file ='wA_eval_20_split_images.pkl'
uA_file ='uA_eval_20_split_images.pkl'

## Visualizing Within Alphabet Pairs

In [ ]:
visualize_n_way(wA_file,n_way = 20)


## Visualizing Unstructured Alphabet Pairs

In [ ]:
visualize_n_way(uA_file,n_way = 20)
